## csvの読み込みと編集

In [2]:
import pandas as pd

In [3]:
# csvファイルの読み込み
#df = pd.read_csv('/home/taki/Otter/rerational.csv')
df = pd.read_csv('/home/taki/Otter/data_for_otter/rerational_k-y.csv')
df

,image_name,path,label,rational_en,basename
0,001_above00230.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,NG,Because there is little scorching on the egg w...,above00230.JPG
1,002_above00034.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,NG,"Because the yolk is round and clean, but the y...",above00034.JPG
2,003_above01063.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,OK,"Because the yolk is round, clean, and soft-coo...",above01063.JPG
3,004_above00979.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,OK,"Because the yolk is off-center but is round, c...",above00979.JPG
4,005_above01079.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,NG,"Because the yolk is round, clean, and soft-coo...",above01079.JPG
...,...,...,...,...,...
195,046_above00618.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,OK,"Because the yolk is round, clean, and soft-boi...",above00618.JPG
196,047_above00642.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,OK,"Because the yolk is round, clean, and soft-boi...",above00642.JPG
197,048_above00043.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,NG,Because there is little scorching on the egg w...,above00043.JPG
198,049_above00973.JPG,C:\Users\taki\katlab\egg\dataset\egg_questiond...,OK,"Because the yolk is round, clean, and soft-boi...",above00973.JPG


In [4]:
num = df.index[df['basename'] == 'above00230.JPG'].values[0]
df["rational_en"][num]


'Because there is little scorching on the egg white, but the yolk is broken.'

In [5]:
print(f'Yes, {df["rational_en"][num]}')

Yes, Because there is little scorching on the egg white, but the yolk is broken.


# データのフォルダの仕分け

## train：val：test  = 100：50：50 に分ける

In [14]:
import os
import shutil
import random

# ディレクトリのパスを指定
# src_dir = './dataset'
# train_dir = './0_train/'
# val_dir = './0_valtest_set/'

src_dir = './0_valtest_set'
train_dir = './0_val/'
val_dir = './0_test/'

# trainとvalの割合
ratio = 0.5

# trainとvalのディレクトリを作成
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# src_dir内の各フォルダに対して
for root, dirs, files in os.walk(src_dir):
    print(f"Current directory: {root}")
    
    # サブディレクトリのリストを表示
    print(f"Subdirectories: {dirs}")
    
    # ファイルのリストを表示
    print(f"Files: {files}")
    # 画像のリストを取得し、シャッフル
    images = [f for f in files if os.path.isfile(os.path.join(root, f))]
    random.shuffle(images)
    
    # trainとvalに分割
    train_images = images[:int(ratio * len(images))]
    val_images = images[int(ratio * len(images)):]
    
    # trainとvalのサブディレクトリを作成
    train_folder = os.path.join(train_dir, os.path.relpath(root, src_dir))
    val_folder = os.path.join(val_dir, os.path.relpath(root, src_dir))
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    
    # 画像をコピー
    for img in train_images:
        shutil.copy(os.path.join(root, img), os.path.join(train_folder, img))
    for img in val_images:
        shutil.copy(os.path.join(root, img), os.path.join(val_folder, img))

print('Images copied to train and val folders.')

Current directory: ./0_valtest_set
Subdirectories: ['egg']
Files: []
Current directory: ./0_valtest_set/egg
Subdirectories: ['egg', 'egg_ng']
Files: []
Current directory: ./0_valtest_set/egg/egg
Subdirectories: []
Files: ['above00786.JPG', 'above00021.JPG', 'above00738.JPG', 'above00351.JPG', 'above00401.JPG', 'above01091.JPG', 'above00386.JPG', 'above01158.JPG', 'above01132.JPG', 'above00225.JPG', 'above01079.JPG', 'above00762.JPG', 'above00961.JPG', 'above00819.JPG', 'above00199.JPG', 'above01041.JPG', 'above01225.JPG', 'above01015.JPG', 'above00957.JPG', 'above00295.JPG', 'above00796.JPG', 'above00979.JPG', 'above00314.JPG', 'above00618.JPG', 'above00020.JPG', 'above00892.JPG', 'above00488.JPG', 'above00896.JPG', 'above01023.JPG', 'above00523.JPG', 'above00504.JPG', 'above00500.JPG', 'above00748.JPG', 'above01139.JPG', 'above00973.JPG', 'above01036.JPG', 'above00505.JPG', 'above01194.JPG', 'above00776.JPG', 'above01048.JPG', 'above01196.JPG', 'above00808.JPG', 'above00319.JPG', 'abo

## context : query = (1 - ratio) : 1 に分ける 

In [17]:
import os
import shutil
import random

# trainとvalでそれぞれ実行する
ratio = 0.8
train_flg = False

# ディレクトリのパスを指定
# src_dir = './0_train'
# train_context_dir = './0_train_context/'
# train_query_dir = './0_train_query/'

# src_dir = './0_val'
# train_context_dir = './0_val_context/'
# train_query_dir = './0_val_query/'

src_dir = './0_test'
train_context_dir = './0_test_context/'
train_query_dir = './0_test_query/'

# train_contextとtrain_queryのディレクトリを作成
os.makedirs(train_context_dir, exist_ok=True)
os.makedirs(train_query_dir, exist_ok=True)

# src_dir内の各フォルダに対して
for root, dirs, files in os.walk(src_dir):
    # 画像のリストを取得し、シャッフル
    images = [f for f in files if os.path.isfile(os.path.join(root, f))]
    random.shuffle(images)
    
    if train_flg:
        # train_contextとtrain_queryに分割
        train_query_images = images[:int(len(images)*ratio)]
        train_context_images  = images[int(len(images)*ratio):]
    else:
        # train_contextとtrain_queryに分割
        train_context_images = images[:5]
        train_query_images  = images[5:]
    
    # train_contextとtrain_queryのサブディレクトリを作成
    train_context_folder = os.path.join(train_context_dir, os.path.relpath(root, src_dir))
    train_query_folder = os.path.join(train_query_dir, os.path.relpath(root, src_dir))
    os.makedirs(train_context_folder, exist_ok=True)
    os.makedirs(train_query_folder, exist_ok=True)
    
    # 画像をコピー
    for img in train_context_images:
        shutil.copy(os.path.join(root, img), os.path.join(train_context_folder, img))
    for img in train_query_images:
        shutil.copy(os.path.join(root, img), os.path.join(train_query_folder, img))

print('Images copied to train_context and train_query folders.')

Images copied to train_context and train_query folders.


context, queryの一方にしか存在しないフォルダがないか確認

In [18]:
import os

def diff(dir1, dir2):
    # train_contextとtrain_queryのサブディレクトリを取得
    folders1 = [os.path.relpath(root, dir1) for root, _, _ in os.walk(dir1)]
    folders2 = [os.path.relpath(root, dir2) for root, _, _ in os.walk(dir2)]

    # train_contextに存在し、train_queryに存在しないフォルダを出力
    unique_folders = set(folders1) - set(folders2)
    print(f"{dir1}に存在し、{dir2}に存在しないフォルダ:")
    for folder in unique_folders:
        print(folder)
        
# ディレクトリのパスを指定
train_context_dir = './0_train_context/'
train_query_dir = './0_train_query/'

diff(train_context_dir, train_query_dir)
diff(train_query_dir, train_context_dir)

train_context_dir = './0_val_context/'
train_query_dir = './0_val_query/'

diff(train_context_dir, train_query_dir)
diff(train_query_dir, train_context_dir)

./0_train_context/に存在し、./0_train_query/に存在しないフォルダ:
./0_train_query/に存在し、./0_train_context/に存在しないフォルダ:
./0_val_context/に存在し、./0_val_query/に存在しないフォルダ:
./0_val_query/に存在し、./0_val_context/に存在しないフォルダ:


## visual_inspection_instructions.jsonの作成　（train, val両方やる）

### jsonのデータ構造生成　コンテキストを埋める

In [29]:
import os
import json
# コンテキストのフォルダを先に実行すること

def natural_sort_key(s):
    """
    ファイル名の数字部分を考慮してソートするためのキー関数
    """
    import re
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

def generate_json_from_directory(directory_path, output_json_path): #コンテキスト用
    output = {"data": {}}
    
    # メインディレクトリ内のサブディレクトリを走査
    for main_folder in os.listdir(directory_path):
        main_folder_path = os.path.join(directory_path, main_folder)
        
        # サブディレクトリがディレクトリであるかの確認
        if os.path.isdir(main_folder_path):
            
            # サブディレクトリ内のサブディレクトリを走査
            for sub_folder in os.listdir(main_folder_path):
                sub_folder_path = os.path.join(main_folder_path, sub_folder)
                
                # サブディレクトリ内のファイルを昇順に走査
                for image_file in sorted(os.listdir(sub_folder_path), key=natural_sort_key):
                    # 画像ファイルの拡張子を除去
                    image_name_without_extension = os.path.splitext(image_file)[0]
                    
                    # キーの名前を生成
                    key_name = f"{main_folder}+{sub_folder}+{image_name_without_extension}"
                    
                    # JSONのデータ構造を生成
                    output["data"][key_name] = {
                        "instruction": "",
                        "answer": "",
                        "flag": "context",
                        "image_ids": [key_name],
                        "rel_ok_ins_ids": [],
                        "rel_ng_ins_ids": []
                    }
    
    # JSONをファイルに書き出し
    with open(output_json_path, "w", encoding="utf-8") as json_file:
        json.dump(output, json_file, indent=4, ensure_ascii=False)

# プログラムの実行
# context_path = "/home/taki/Otter/0_train_context"
# output_json_path = "train_VI_instructions.json"
# context_path = "/home/taki/Otter/0_val_context"
# output_json_path = "val_VI_instructions.json"
context_path = "/home/taki/Otter/0_test_context"
output_json_path = "test_VI_instructions.json"

generate_json_from_directory(context_path, output_json_path)

### jsonのデータ構造生成　クエリを埋める

In [30]:
import os
import json
import random
# コンテキストのフォルダを実行した後に実行すること

def natural_sort_key(s):
    """
    ファイル名の数字部分を考慮してソートするためのキー関数
    """
    import re
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

def generate_and_merge_json_from_directory(directory_path, old_json_path, NUM): #クエリ用
    # 以前のJSONを読み込む
    with open(old_json_path, "r", encoding="utf-8") as f:
        old_data = json.load(f)

    new_data = {"data": {}}

    # メインディレクトリ内のサブディレクトリを走査
    for main_folder in os.listdir(directory_path):
        main_folder_path = os.path.join(directory_path, main_folder)

        # サブディレクトリがディレクトリであるかの確認
        if os.path.isdir(main_folder_path):

            # サブディレクトリ内のサブディレクトリを走査
            for sub_folder in os.listdir(main_folder_path):
                sub_folder_path = os.path.join(main_folder_path, sub_folder)

                # サブディレクトリ内のファイルを昇順に走査
                for image_file in sorted(os.listdir(sub_folder_path), key=natural_sort_key):
                    # 画像ファイルの拡張子を除去
                    image_name_without_extension = os.path.splitext(image_file)[0]

                    # キーの名前を生成
                    key_name = f"{main_folder}+{sub_folder}+{image_name_without_extension}"

                    # rel_ok_ins_idsの値を取得
                    related_ok_keys = [key for key in old_data["data"].keys() if f"{main_folder}+{main_folder}+" in key]
                    rel_ok_ins_ids = random.sample(related_ok_keys, min(NUM, len(related_ok_keys)))

                    # rel_ng_ins_idsの値を取得
                    if main_folder == sub_folder:
                        related_ng_keys = [key for key in old_data["data"].keys() if f"{main_folder}+" in key and f"{main_folder}+{main_folder}+" not in key]
                        related_ng_key_lists = {}
                        for key in related_ng_keys:
                            prefix = key.split('+')[1]
                            if prefix not in related_ng_key_lists:
                                related_ng_key_lists[prefix] = []
                            related_ng_key_lists[prefix].append(key)

                        rel_ng_ins_ids = []
                        while len(rel_ng_ins_ids) < NUM and related_ng_key_lists:
                            for prefix in list(related_ng_key_lists.keys()):
                                if related_ng_key_lists[prefix]:
                                    selected_key = random.choice(related_ng_key_lists[prefix])
                                    rel_ng_ins_ids.append(selected_key)
                                    related_ng_key_lists[prefix].remove(selected_key)
                                else:
                                    del related_ng_key_lists[prefix]
                    else:
                        related_ng_keys = [key for key in old_data["data"].keys() if f"{main_folder}+{sub_folder}+" in key]
                        rel_ng_ins_ids = random.sample(related_ng_keys, min(NUM, len(related_ng_keys)))

                    # JSONのデータ構造を生成
                    new_data["data"][key_name] = {
                        "instruction": "",
                        "answer": "",
                        "flag": "query",
                        "image_ids": [key_name],
                        "rel_ok_ins_ids": rel_ok_ins_ids,
                        "rel_ng_ins_ids": rel_ng_ins_ids
                    }

    # 既存のJSONデータに新しいデータをマージ
    old_data["data"].update(new_data["data"])

    # JSONをファイルに書き出し
    # old_json_path = "output.json"
    
    with open(old_json_path, "w", encoding="utf-8") as json_file:
        json.dump(old_data, json_file, indent=4, ensure_ascii=False)

# プログラムの実行
# query_path = "/home/taki/Otter/0_train_query"
# json_path = "train_VI_instructions.json"
# query_path = "/home/taki/Otter/0_val_query"
# json_path = "val_VI_instructions.json"
query_path = "/home/taki/Otter/0_test_query"
json_path = "test_VI_instructions.json"
NUM = 5 # 良品、不良品をそれぞれ5枚ずつ選ぶ
generate_and_merge_json_from_directory(query_path, json_path, NUM)

In [31]:
# 数の確認
json_path = "train_VI_instructions.json"
json_path = "val_VI_instructions.json"
json_path = "test_VI_instructions.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)["data"]
    
entries = [item for item in data.values() if item["flag"] == "context"]
count = len(entries)
print(f"context: {count}")
    
entries = [item for item in data.values() if item["flag"] == "query"]
count = len(entries)
print(f"query: {count}")

context: 10
query: 40


## instructionとanswerを埋める

In [32]:
import json
import os

# 指定されたリスト形式の文字列を作成する関数
def generate_list_string(items):
    # アンダースコアをスペースに変換
    items = [item.replace('_', ' ') for item in items]
    
    if len(items) == 1:
        return items[0]
    elif len(items) == 2:
        return f"{items[0]} and {items[1]}"
    else:
        return ", ".join(items[:-1]) + f", and {items[-1]}"

def fill_instruction_and_answer(json_path, train_context_dir):
    # JSONを読み込む
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)["data"]

    # ['data']からキーを順に読み込む
    for key, value in data.items():
        # キーの名前から親フォルダ名と子フォルダ名を認識する
        parent_folder, child_folder, image_name = key.split("+")

        # "./1_train_context/親フォルダ名"にアクセスし、その中にある子フォルダ名をリスト化する
        subfolder_list = os.listdir(os.path.join(train_context_dir, parent_folder))
        subfolder_list = [folder for folder in subfolder_list if folder != parent_folder] # 親フォルダ名を除外

        # "instruction"にテキストを入力する
        parent_folder__ = parent_folder.replace('_', ' ')
        value["instruction"] = f'Is this image the preferred fried egg? The preferred fried egg is determined by the following criteria: the fried egg should not be over-burnt, the yolk should not be crushed, the yolk should not be overcooked and dry, there should not be too much oil, and the white should not be raw and cooked to perfection.'
        
        
        ans_txt =df["rational_en"][df.index[df['basename'] == f'{image_name}.JPG'].values[0]] # image_nameと一致する根拠を取得
        # "answer"にテキストを入力する
        if parent_folder == child_folder: # 良品
            value["answer"] = f'Yes, {ans_txt}'
        else: # 不良品
            value["answer"] = f'No, {ans_txt}'

    # JSONをファイルに書き出し
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump({"data": data}, json_file, indent=4, ensure_ascii=False)

# # プログラムの実行
# json_path = "train_VI_instructions.json"
# train_context_dir = "./0_train_context" # コンテキストのみでok (クエリと同じフォルダ構成のため)
# json_path = "val_VI_instructions.json"
# train_context_dir = "./0_val_context" # コンテキストのみでok (クエリと同じフォルダ構成のため)
json_path = "test_VI_instructions.json"
train_context_dir = "./0_test_context" # コンテキストのみでok (クエリと同じフォルダ構成のため)
fill_instruction_and_answer(json_path, train_context_dir)

## visual_inspection_train.jsonの作成

In [39]:
import json
import random

def create_visual_inspection_train(input_json_path, output_json_path, NUM_PAIRS):
    # JSONを読み込む
    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)["data"]

    train_data = {}

    # ['data']からキーを順に読み込む
    for key, value in data.items():
        # 'flag'=='query'のキーを選択
        if value["flag"] == "query":
            # "rel_ok_ins_ids"と"rel_ng_ins_ids"を読み込み
            ok_ids = value["rel_ok_ins_ids"]
            ng_ids = value["rel_ng_ins_ids"]

            # すべての組み合わせをリストに保存
            combinations = []
            for ok_id in ok_ids:
                for ng_id in ng_ids:
                    combinations.append([ok_id, ng_id])

            # リスト内の各組内の要素をランダムにシャッフル
            for combination in combinations:
                random.shuffle(combination)

            # キー名を更新してデータを保存
            random.shuffle(combinations)
            for i, combination in enumerate(combinations[:NUM_PAIRS]):
                train_data[f"{key}={i}"] = combination


    # JSONをファイルに書き出し
    with open(output_json_path, "w", encoding="utf-8") as json_file:
        json.dump(train_data, json_file, indent=4, ensure_ascii=False)

# プログラムの実行
NUM_PAIRS = 1 # 1つのクエリに対して生成する組み合わせの数:Val,testは1（クエリが重複してほしくない）.Trainは5
input_json_path = "test_VI_instructions.json"
output_json_path = f"test_VI_pairs{NUM_PAIRS}_train.json"
create_visual_inspection_train(input_json_path, output_json_path, NUM_PAIRS)

In [40]:
# 数の確認
json_path = "test_VI_pairs1_train.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)
    
count = len(data.keys())
print(f"ペア数: {count}")

ペア数: 40


## visual_inspection.jsonの作成

In [47]:
import json
import os
from PIL import Image
import io
import base64

# コンテキスト　→　クエリの順で実行すること

def image_to_urlsafe_base64_png(img_path):
    """画像をメモリ上でPNGに変換し、その後URL-safeなBase64に変換する関数"""
    with Image.open(img_path) as image:
        # CMYKモードの画像をRGBモードに変換
        if image.mode == 'CMYK':
            image = image.convert('RGB')
        # パレットモードの画像をRGBAモードに変換
        if image.mode == 'P':
            image = image.convert('RGBA')
        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.urlsafe_b64encode(buffered.getvalue()).decode('utf-8')
    return img_str

def create_visual_inspection(input_json_path, base_folder, output_json_path):
    # JSONを読み込む
    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)["data"]

    # 既存のvisual_inspection.jsonが存在する場合、その内容を読み込む
    if os.path.exists(output_json_path):
        with open(output_json_path, "r", encoding="utf-8") as f:
            visual_data = json.load(f)
    else:
        visual_data = {}

    extensions = ['.png', '.jpg', '.jpeg', '.JPG']

    # ['data']からキーを順に読み込む
    for key in data.keys():
        parent_folder, child_folder, image_name = key.split("+")
        
        # 各拡張子を試して、存在するファイルを見つける
        for ext in extensions:
            img_path = os.path.join(base_folder, parent_folder, child_folder, image_name + ext)
            if os.path.exists(img_path):
                # 画像をURL-safeなBase64 PNG形式に変換
                visual_data[key] = image_to_urlsafe_base64_png(img_path)
                break

    # JSONをファイルに書き出し
    with open(output_json_path, "w", encoding="utf-8") as json_file:
        json.dump(visual_data, json_file, indent=4, ensure_ascii=False)

# プログラムの実行
phase_name = "test"
base_folder = f"./0_{phase_name}_context"  # これは一つ上のベースフォルダのパス
input_json_path = f"{phase_name}_VI_instructions.json"
output_json_path = f"{phase_name}_VI.json"
create_visual_inspection(input_json_path, base_folder, output_json_path)

base_folder = f"./0_{phase_name}_query"  # これは一つ上のベースフォルダのパス
input_json_path = f"{phase_name}_VI_instructions.json"
output_json_path = f"{phase_name}_VI.json"
create_visual_inspection(input_json_path, base_folder, output_json_path)

In [48]:
import json

# 数の確認
json_path = f"{phase_name}_VI.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)
    
count = len(data.keys())
print(f"画像枚数: {count}")

画像枚数: 50


# queryのみのpairs{i}_train.jsonを作成

In [5]:
import json
import random

def create_visual_inspection_train(input_json_path, output_json_path, NUM_PAIRS):
    # JSONを読み込む
    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)["data"]

    train_data = {}

    # ['data']からキーを順に読み込む
    for key, value in data.items():
        train_data[f"{key}=0"] = []


    # JSONをファイルに書き出し
    with open(output_json_path, "w", encoding="utf-8") as json_file:
        json.dump(train_data, json_file, indent=4, ensure_ascii=False)

# プログラムの実行
NUM_PAIRS = 1 # 1つのクエリに対して生成する組み合わせの数:Val,testは1（クエリが重複してほしくない）.Trainは5
input_json_path = "/home/taki/Otter/data_for_otter/data/training_data/0/test_VI_instructions.json"
output_json_path = f"/home/taki/Otter/data_for_otter/data/training_data/0/test_VI_query-only_train.json"
create_visual_inspection_train(input_json_path, output_json_path, NUM_PAIRS)

In [3]:
input_json_path = "/home/taki/Otter/data_for_otter/data/training_data/0/train_VI_instructions.json"
with open(input_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)["data"]

data

{'egg+egg+above00161': {'instruction': 'Is this image the preferred fried egg? The preferred fried egg is determined by the following criteria: the fried egg should not be over-burnt, the yolk should not be crushed, the yolk should not be overcooked and dry, there should not be too much oil, and the white should not be raw and cooked to perfection.',
  'answer': 'Yes, because there are no issues with the judging criteria, and the yolk is semi-cooked.',
  'flag': 'context',
  'image_ids': ['egg+egg+above00161'],
  'rel_ok_ins_ids': [],
  'rel_ng_ins_ids': []},
 'egg+egg+above00735': {'instruction': 'Is this image the preferred fried egg? The preferred fried egg is determined by the following criteria: the fried egg should not be over-burnt, the yolk should not be crushed, the yolk should not be overcooked and dry, there should not be too much oil, and the white should not be raw and cooked to perfection.',
  'answer': 'Yes, because the white around the yolk appears undercooked, but othe